<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# загрузим потенциально нужные библиотеки
import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# загрузим данные
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('insurance.csv')

In [3]:
# проверим данные на всякое нехорошее. использую функцию, которой пользовался в прошлых проектах
names= [df]

def info_func(names):
    for name in names:
        df_temp = pd.DataFrame(name)
        display(df_temp.head(10))
        display(df_temp.info())
        display('Кол-во дублей:', df_temp.duplicated().sum())
        display('Кол-во НАНов:', df_temp.isna().sum())
        display('Статистика:', df_temp.describe())
    return
info_func(names)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

'Кол-во дублей:'

153

'Кол-во НАНов:'

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

'Статистика:'

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [4]:
# дубли видим, они на не нужны
df.drop_duplicates(inplace=True)

На этом предлагаю закночить предобработку данных, т.к. кроме дублей тут особо мы ничего скорее всего не найдём. Особых выбросов нет, типы данных тоже в порядке

## Умножение матриц

**Внимание, вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:** 
Чтобы обосновать ответ, будем просто рассжудать, пусть: 
$$
X' = XP
$$
тогда
$$w' = ((XP)^T XP)^{-1} XP^T y = (P^T X^T XP)^{-1}P^{T^{-1}}P^T X^T y = P^{-1}(X^T X)^{-1}P^{T^{-1}}P^T X^T y$$
т.к.
$$
(P^T)^{1}P^T = E
$$ -  Матрица умножается на обратную и становится единичной
то результат нашего послежнего преобразования можно записать как
$$
w' = P^{-1} w 
$$
тогда
$$
a' = XPw' = XPP^{-1}w = XEw = Xw = a 
$$

## Алгоритм преобразования

**Алгоритм**
В качестве алгоритма преобразования можно домножить обучающие признаки X на случайную обратимую матрицу P размером 4х4, т.к. по правилу умножения матриц, количество столбцов в них должно быть равным, а по условию обратимости - матрица должна быть квадратной. Далее обучим модели линейной регрессии на данных до преобразования и после. Метрики R2 на обеих моделях должны быть равны. 

**Обоснование**
Как было доказано в предыдущем пункте, домножение признаков на какую-либо обратимую матрицу не приводит к изменению предсказаний, поэтому результаты R2 должны оказаться равны. Пора переходить к практике

## Проверка алгоритма

In [5]:
# сначала объявим переменные признаков
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [6]:
# сразу же обучим исходную модель и посчитаем R2
model1 = LinearRegression()
model1.fit(features, target)
predictions = model1.predict(features)
r2_1 = r2_score(target, predictions)
print(f'Метрика R2 для признаков без шифрования {r2_1}')

Метрика R2 для признаков без шифрования 0.4302010044852067


Теперь нужно создать случайную обратимую матрицу P, чтобы наша теория сработала

In [7]:
det = 0 # переменная для определителя, 0 - матрица необратимая, неравно 0 - обратимая
P = 0 # матрица
# цикл, который будет выполняться до тех пор, пока не выполнится наше условие
while det == 0:
    P = np.random.normal(size=(4, 4)) # генерируем случайную матрицу
    det = np.linalg.det(P) # проверяем на обратимость
P # возвращаем матрицу если профит

array([[-0.66917203,  1.43727562, -0.36083052,  0.33834175],
       [-0.34624381, -1.10472343, -0.51145414, -1.30172239],
       [ 0.22582749, -0.03350771,  0.05018376,  1.07457644],
       [-1.41933289,  1.12405075,  0.42548685, -0.18098504]])

Полдела сделано, осталось проверить как работает

In [8]:
# домножим наши признаки на вновь созданную матрицу
features_2 = features @ P

# сразу обучим модель
model_2 = LinearRegression()
model_2.fit(features_2, target)
predictions_2 = model_2.predict(features_2)
r2_2 = r2_score(target, predictions_2)
print(f'Метрика R2 для признаков с шифрованием {r2_2}')

Метрика R2 для признаков с шифрованием 0.4302010044852064


### Выводы

По итогу работы можно сделать следующие выводы:
* Умножение признаков на обратимую матрицу, действительно не портит качество модели линейной регрессии. Хотя, если делать сравнение метрик до самого последнего знака после запятой, то разница есть, но, очевидно, ей можно принебречь
* Вероятно, как-то так и работает Scaler. На предыдущих проектах было выяснено, что масштабирование не влияет на метрики качества линейной регрессии и на метрики лесов

Значит, данный метод можно использовать не только в практике обучения, но и в практике обезличенного хранения информации, успевай только матрицы случайные генерировать... (Это не единственный метод защиты данных, но, кажется, Яндекс.Практикум и Яндекс.Еда об этом не знали. Надо бы соответсвующий курс в прайс добавить...)